In [ ]:
from google_play_scraper import reviews, Sort
import pandas as pd
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import cx_Oracle

# Download required NLTK resources
nltk.download('stopwords')
nltk.download('vader_lexicon')

# Load English NLP and stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = set(stopwords.words('english'))
vader = SentimentIntensityAnalyzer()

# Step 1: Fetch reviews from Google Play
def fetch_reviews(app_id, bank_name, count=400):
    all_reviews = []
    for _ in tqdm(range(0, count, 100)):
        result, _ = reviews(
            app_id,
            lang='en',
            country='et',
            sort=Sort.NEWEST,
            count=100
        )
        for r in result:
            all_reviews.append({
                "review": r['content'],
                "rating": r['score'],
                "date": r['at'].strftime('%Y-%m-%d'),
                "bank": bank_name,
                "source": "Google Play"
            })
    return all_reviews

# Bank app package IDs
apps = {
    "com.combanketh.mobilebanking": "CBE",
    "com.boa.boaMobileBanking": "BOA",
    "com.cr2.amolelight": "Dashen"
}

# Fetch and save raw reviews
all_data = []
for package, name in apps.items():
    all_data += fetch_reviews(package, name, 400)

df = pd.DataFrame(all_data)
df.to_csv("raw_reviews.csv", index=False)

# Step 2: Clean and deduplicate data
df.drop_duplicates(subset=["review", "bank"], inplace=True)
df.dropna(subset=["review", "rating", "date", "bank", "source"], inplace=True)
df.to_csv("cleaned_reviews.csv", index=False)



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
100%|██████████| 263/263 [00:00<00:00, 3704.29it/s]


Top Keywords by Bank:
CBE: ['app', 'good', 'nice', 'like', 'network', 'make', 'use', 'bank', 'work', 'problem', 'awesome', 'cbe', 'fast', 'love', 'service', 'issue', 'fantastic', 'money', 'banking', 'send']
BOA: ['good', 'app', 'work', 'boa', 'bank', 'bad', 'like', 'use', 'really', 'open', 'mobile', 'wow', 'thank', 'excellent', 'application', 'update', 'time', 'great', 'make', 'nice']
Dashen: ['good', 'app', 'nice', 'use', 'banking', 'work', 'wow', 'amole', 'bank', 'mobile', 'update', 'dashin', 'need', 'fast', 'slow', 'dashen', 'love', 'like', 'option', 'bad']

✅ Pipeline finished. Results saved to 'bank_reviews_analysis.csv'


In [10]:
import cx_Oracle

# --- Database Connection Setup ---
dsn = cx_Oracle.makedsn("localhost", 1522, service_name="XE")
conn = cx_Oracle.connect(user="system", password="system", dsn=dsn)
cursor = conn.cursor()

# --- Create Table (if not exists) ---
cursor.execute("""
BEGIN
   EXECUTE IMMEDIATE 'CREATE TABLE banks (
       bank_id NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
       bank_name VARCHAR2(100) UNIQUE
   )';
EXCEPTION
   WHEN OTHERS THEN
      IF SQLCODE != -955 THEN -- ORA-00955: name is already used by an existing object
         RAISE;
      END IF;
END;
""")

# --- Sample Bank List ---
banks = [ "Dashen Bank", "CBE",  "Abyssinia Bank"]

# --- Insert Banks and Get IDs ---
bank_id_map = {}

for bank in banks:
    bank_id_var = cursor.var(int)
    try:
        cursor.execute(
            "INSERT INTO banks (bank_name) VALUES (:1) RETURNING bank_id INTO :2",
            [bank, bank_id_var]
        )
        bank_id_map[bank] = bank_id_var.getvalue()[0]
    except cx_Oracle.IntegrityError:
        # Bank already exists, fetch its ID
        cursor.execute("SELECT bank_id FROM banks WHERE bank_name = :1", [bank])
        bank_id_map[bank] = cursor.fetchone()[0]

# --- Commit changes ---
conn.commit()

# --- Print Result ---
print("Inserted or existing banks with IDs:")
for name, id in bank_id_map.items():
    print(f"{name}: {id}")

# --- Cleanup ---
cursor.close()
conn.close()


Inserted or existing banks with IDs:
Dashen Bank: 4
CBE: 5
Abyssinia Bank: 7
